# Train YOLOv8 with Data Augmentation

In [27]:
import os
import argparse
import logging
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import pandas as pd
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
import cv2

# Custom Dataset Class to Load the Data

In [28]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, label_dir, transform=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.transform = transform
        self.images = [img for img in os.listdir(image_dir) if img.endswith('.jpg')]
        self.images.sort()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.image_dir, img_name)
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label_path = os.path.join(self.label_dir, img_name.replace('.jpg', '.txt'))
        boxes = pd.read_csv(label_path, header=None, delim_whitespace=True).values

        if self.transform:
            augmented = self.transform(image=image, bboxes=boxes)
            image = augmented['image']
            boxes = augmented['bboxes']

        return image, boxes

# Augmentations

In [29]:
def get_transforms():
    return A.Compose([
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Transpose(p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HueSaturationValue(p=0.5),
        A.GaussianBlur(p=0.3),
        A.GaussNoise(p=0.3),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.RandomSizedBBoxSafeCrop(640, 640, p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params=A.BboxParams(format='yolo', label_fields=[]))

# Training with Augmented Data (on-the-fly)

In [30]:
def train_yolo(model_name, data_path, total_epochs, img_size):
    # Load model
    model = YOLO(model_name)

    # creating datasets and loaders
    train_transforms = get_transforms()
    train_dataset = CustomDataset('/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/train', 
                                  '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/train/labels', 
                                  transform=train_transforms)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

    val_transforms = get_transforms()
    val_dataset = CustomDataset('/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/valid/images', 
                                '/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/valid/labels', 
                                transform=val_transforms)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

    # Training
    results = model.train(
        train_loader=train_loader,
        val_loader=val_loader,
        epochs=total_epochs,
        imgsz=img_size,
        save=True,
        plots=True
    )

In [31]:
def parse_args():
    parser = argparse.ArgumentParser(description="Train YOLOv8 model.")
    parser.add_argument('--model', type=str, default='yolov8s.pt', help='Pre-trained model path')
    parser.add_argument('--data', type=str, required=True, help='Path to dataset')
    parser.add_argument('--epochs', type=int, default=25, help='Number of epochs to train')
    parser.add_argument('--img_size', type=int, default=800, help='Image size for training')
    return parser.parse_args()

def parse_args_notebook():
    args = argparse.Namespace(
        model='yolov8s.pt',
        data='/home/jupyter/ee_tree_counting/Data/Dataset 348-17-15/data.yaml',
        epochs=100,
        img_size=640,
        batch=16,
        save=True,
        plots=True
    )
    return args

In [32]:
args = parse_args_notebook()
train_yolo(args.model, args.data, args.epochs, args.img_size)

ValueError: num_samples should be a positive integer value, but got num_samples=0

# Data Augmentation: Geometric Augmentation Mainly

### Plan: See if geometric augmentations improve anything, then build big dataset. Do augmentations and add that as further training. Then integrate annotated satellite images with seasonal augmentation, enhanced contrast etc.